In [1]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask.grid import *

In [2]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [3]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

Done applying negbuff of -30 and filtering small labels of area less than 3600


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [ ]:
img_paths, label_paths = wflow.grid_images()

In [ ]:
wflow.remove_mostly_empty(img_paths, label_paths)

wflow.move_chips_to_folder()

In [ ]:
import time
start = time.time()
grid_images_rasterio_controlled_processes('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/mnt/point/landsat-1024-cp/landsat_grid_imgs', MAX_PROCESSES = 11)
stop = time.time()
print(stop-start)

4
4


In [1]:
import rasterio
from cropmask.grid import *
src = rasterio.open('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', shared=False)

In [2]:
chip_list = get_tiles_for_threaded_map(src, width=512, height=512)

In [3]:
window, transform = chip_list[0]

In [6]:

src.meta['transform'] = transform
src.meta['width'], src.meta['height'] = window.width, window.height

In [12]:
import time
start = time.time()
# this makes it take 27 seconds, not the windows reading
with rasterio.open("/home/ryan/test.tif", 'w', **src.meta, shared=False) as outds: 
    outds.write(arr)
stop = time.time()
print(stop-start)

1.9120993614196777


In [20]:
arr.shape

(3, 512, 512)